In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import tensorflow as tf
import numpy as np
from tqdm import tqdm, trange
import os, sys
sys.path.append(os.getcwd())

Populating the interactive namespace from numpy and matplotlib


In [5]:
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from helper.feature_extraction import *
from helper.sequence_labeling import *
from helper.transformation import *

In [6]:
inputs = Input(shape=(320,320,3), name='inputs')

In [7]:
gg = GridGenerator(20, (inputs.shape[1], inputs.shape[2]))

TypeError: __init__() missing 1 required positional argument: 'value_index'

In [ ]:
# the actual loss calc occurs here despite it not being
# an internal Keras loss function

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def net(input_shape=(320, 320, 3), hidden=256, output_size=64, num_classes=1000, debug=False):
    inputs = Input(shape=input_shape, name='inputs')
    # STN started here
    I_r = tps_stn(F=20, I_size=(inputs.shape[1], inputs.shape[2]),I_r_size=(inputs.shape[1], inputs.shape[2]),I_channels=1)(inputs)
    # reshape to [batch, w, h*dims], already included in bisequence
    # feeds through resnet50v2
    res_out = resnet50v2(input_tensor=I_r, classes=num_classes).out
    # added biLSTM here, consider between LSTM and GRU
    lstm_out = bisequence(res_out, hidden, output_size, num_classes=num_classes).output
    infer = Model(inputs=inputs, outputs=out, name=f'infer_{input_shape[0]}')
    # added ctc_lost
    labels = Input(name='label', shape=[label_len], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')

    loss = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([out, labels, input_length, label_length])
    train = Model(inputs=[inputs, labels, input_length, label_length], outputs=[loss], name=f'train_{input_shape[0]}')
    return train, infer